In [2]:
import json
import numpy as np
import pandas as pd
import re

from pathlib import Path

In [25]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True)
df_annotations = pd.read_json("./results/annotation_subset.json")
df = df.join(df_annotations.set_index('pairID'), on='pairID')
df = df[df.label.isna()]

In [32]:
new_df = df[df['style'] == 'pir'].sample(12)
new_df

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,...,sentence2_transform2,sentence1_transform1_bertscore,sentence1_transform2_bertscore,sentence2_transform1_bertscore,sentence2_transform2_bertscore,text,label,orig_prem,orig_hyp,orig_label
512,"[entailment, entailment, entailment, entailmen...",travel,entailment,69141e,69141,It was replaced in 1910 by the famous old pont...,( It ( ( was ( ( replaced ( in 1910 ) ) ( by (...,(ROOT (S (NP (PRP It)) (VP (VBD was) (VP (VBN ...,The famous old pontoon bridge with its seafood...,( ( ( The ( famous ( old ( pontoon bridge ) ) ...,...,Th' notorious ancient pontoon bridge wit' its ...,0.921927,0.926414,0.951812,0.951840,NaN,NaN,NaN,NaN,NaN
2425,"[neutral, entailment, neutral, neutral, neutral]",travel,neutral,53509n,53509,The entrance is also home to several sculpture...,( ( The entrance ) ( ( ( is also ) ( ( ( home ...,(ROOT (S (NP (DT The) (NN entrance)) (VP (VBZ ...,There are many types of art found in the galle...,( There ( ( are ( ( many types ) ( of ( art ( ...,...,Thar be plenty o' kinds o' art found in th' ga...,0.895482,0.912436,0.918516,0.889694,NaN,NaN,NaN,NaN,NaN
1838,"[contradiction, contradiction, contradiction, ...",government,contradiction,4996c,4996,"Also, considerable sums are spent by the Posta...","( Also ( , ( ( ( ( ( ( considerable sums ) ( a...","(ROOT (S (ADVP (RB Also)) (, ,) (S (NP (JJ con...",The Postal Service does not participate in wor...,( ( The ( Postal Service ) ) ( ( ( does not ) ...,...,Thar be no participation in worksharing fer th...,0.869998,0.907231,0.871399,0.884564,NaN,NaN,NaN,NaN,NaN
2044,"[neutral, neutral, contradiction, entailment, ...",slate,neutral,38527n,38527,"will never be doused (Brit Hume, Fox News Sund...",( ( ( will never ) ( ( be doused ) ( -LRB- ( (...,(ROOT (S (S (VP (MD will) (ADVP (RB never)) (V...,They wanted the public to know where the funds...,( They ( ( wanted ( the ( public ( to ( know (...,...,Thar goal be to inform the landlubbers where t...,0.952855,0.957760,0.884973,0.877962,NaN,NaN,NaN,NaN,NaN
550,"[neutral, neutral, neutral, neutral, neutral]",travel,neutral,9128n,9128,At the top of the hill is the imposing medieva...,( ( At ( ( the top ) ( of ( the hill ) ) ) ) (...,(ROOT (S (PP (IN At) (NP (NP (DT the) (NN top)...,"In medieval times, the fortress withstood many...","( ( In ( medieval times ) ) ( , ( ( the fortre...",...,"In th' middle ages, th' stronghold held agains...",0.880341,0.872492,0.895556,0.915761,NaN,NaN,NaN,NaN,NaN
1972,"[contradiction, contradiction, contradiction, ...",government,contradiction,18027c,18027,They said that (1) agencies need to be able to...,( They ( said ( ( that ( -LRB- ( 1 -RRB- ) ) )...,(ROOT (S (NP (PRP They)) (VP (VBD said) (SBAR ...,It was stated that the agencies could ignore c...,( It ( ( was ( stated ( that ( ( the agencies ...,...,'Twas stated that th' agencies could be ignori...,0.921696,0.890427,0.919557,0.912373,NaN,NaN,NaN,NaN,NaN
2669,"[entailment, entailment, neutral, neutral, neu...",fiction,neutral,77175e,77175,"No money no results!"" Another voice which Tomm...",( ( ( ( ( ( ( No ( money no ) ) results ) ! ) ...,(ROOT (S (S (NP (DT No) (NN money) (RB no)) (V...,Can you guarantee that there will be results w...,( ( ( Can you ) ( guarantee ( that ( there ( w...,...,Be ye swearin' on yer planks that there be pro...,0.905854,0.904481,0.827557,0.865674,NaN,NaN,NaN,NaN,NaN
2645,"[entailment, entailment, entailment, neutral, ...",telephone,entailment,98721e,98721,that's neat just supervised more or less than ...,( that ( ( 's neat ) ( ( just ( supervised ( (...,(ROOT (SBAR (WHNP (WDT that)) (S (VP (VBZ 's) ...,They just supervise people.,( They ( just ( ( supervise people ) . ) ) ),...,"They be keepin' an eye on people, matey.",0.808518,0.799391,0.887667,0.868170,NaN,NaN,NaN,NaN,NaN
2264,"[contradiction, contradiction, contradiction, ...",travel,contradiction,95186c,95186,"The cane plantations, in

In [36]:
annotation_set = []

for index, row in new_df.iterrows():
    premise = row['sentence1_transform1'] if row['sentence1_transform1_bertscore'] > row['sentence1_transform2_bertscore'] else row['sentence1_transform2']
    hypothesis = row['sentence2_transform1'] if row['sentence2_transform1_bertscore'] > row['sentence2_transform2_bertscore'] else row['sentence2_transform2']
    annotation_set.append({
        'pairID': row['pairID'],
        'text': f"Premise:\n{premise}\n\nHypothesis:\n{hypothesis}",
        'label': row['gold_label'],
        'orig_prem': row['sentence1'],
        'orig_hyp': row['sentence2'],
        'orig_label': row['gold_label']
    })
    
annotation_df = pd.DataFrame(annotation_set)

In [37]:
save_path = Path('./results/annotation_subset_extra.json')
annotation_df.to_json(save_path, orient="records")

In [14]:
df.groupby('style').size()

style
emb    172
pir    154
sha    174
dtype: int64

In [15]:
df.groupby('genre').size()

genre
fiction        99
government    105
slate         102
telephone     102
travel         92
dtype: int64

In [13]:
df.groupby(['style', 'genre']).size()

style  genre     
emb    fiction       34
       government    37
       slate         37
       telephone     33
       travel        31
pir    fiction       30
       government    30
       slate         36
       telephone     31
       travel        27
sha    fiction       35
       government    38
       slate         29
       telephone     38
       travel        34
dtype: int64

500